# Video Games Sales

This notebook is for data analysis of video game sales.

Data: [Kaggle.com](https://www.kaggle.com/datasets/gregorut/videogamesales)

In [92]:
import pandas as pd
import numpy as np
import pickle
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Load the dataset from local storage and do the basic checks

Firstly, we will load the data into df_main and then we will display some basic information about the data frame - the head and information.

In [2]:
df_main = pd.read_csv('../files/vgsales.csv')
df_main.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


## Missing Values

THere are couple of columns with missing data:
1. Year
2. Publisher

Let's verify for which games we are missing this data. If these are blockbusters, we need to do something about those. But if they are some unknown titles for old platforms, we can drop observations with values missing in these columns

In [4]:
df_main[df_main['Year'].isna()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
179,180,Madden NFL 2004,PS2,NaN,Sports,Electronic Arts,4.26,0.26,0.01,0.71,5.23
377,378,FIFA Soccer 2004,PS2,NaN,Sports,Electronic Arts,0.59,2.36,0.04,0.51,3.49
431,432,LEGO Batman: The Videogame,Wii,NaN,Action,Warner Bros. Interactive Entertainment,1.86,1.02,0.00,0.29,3.17
470,471,wwe Smackdown vs. Raw 2006,PS2,NaN,Fighting,NaN,1.57,1.02,0.00,0.41,3.00
607,608,Space Invaders,2600,NaN,Shooter,Atari,2.36,0.14,0.00,0.03,2.53
...,...,...,...,...,...,...,...,...,...,...,...
16307,16310,Freaky Flyers,GC,NaN,Racing,Unknown,0.01,0.00,0.00,0.00,0.01
16327,16330,Inversion,PC,NaN,Shooter,Namco Bandai Games,0.01,0.00,0.00,0.00,0.01
16366,16369,Hakuouki: Shinsengumi Kitan,PS3,NaN,Adventure,Unknown,0.01,0.00,0.00,0.00,0.01
16427,16430,Virtua Quest,GC,NaN,Role-Playing,Unknown,0.01,0.00,0.00,0.00,0.01


As we can see, there are some titles with significant sale result (like LEGO: BATMAN for Wii or FIFA Soccer 2004 for PS2).

Some are not important, so let's filter for those which have sales bigger than 0.5

In [5]:
df_games_with_missing_year = df_main[(df_main['Year'].isna()) & (df_main['Global_Sales'] > 0.5)]
df_games_with_missing_year

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
179,180,Madden NFL 2004,PS2,NaN,Sports,Electronic Arts,4.26,0.26,0.01,0.71,5.23
377,378,FIFA Soccer 2004,PS2,NaN,Sports,Electronic Arts,0.59,2.36,0.04,0.51,3.49
431,432,LEGO Batman: The Videogame,Wii,NaN,Action,Warner Bros. Interactive Entertainment,1.86,1.02,0.00,0.29,3.17
470,471,wwe Smackdown vs. Raw 2006,PS2,NaN,Fighting,NaN,1.57,1.02,0.00,0.41,3.00
607,608,Space Invaders,2600,NaN,Shooter,Atari,2.36,0.14,0.00,0.03,2.53
624,625,Rock Band,X360,NaN,Misc,Electronic Arts,1.93,0.34,0.00,0.21,2.48
649,650,Frogger's Adventures: Temple of the Frog,GBA,NaN,Adventure,Konami Digital Entertainment,2.15,0.18,0.00,0.07,2.39
652,653,LEGO Indiana Jones: The Original Adventures,Wii,NaN,Action,LucasArts,1.54,0.63,0.00,0.22,2.39
711,713,Call of Duty 3,Wii,NaN,Shooter,Activision,1.19,0.84,0.00,0.23,2.26
782,784,Rock Band,Wii,NaN,Misc,MTV Games,1.35,0.56,0.00,0.20,2.11


There are some significant releases

We need to find release year for them.

I will [Twitch API](https://api-docs.igdb.com/#getting-started) to do it

In [6]:
import os
import requests
import asyncio
import aiohttp
import ssl
from dotenv import load_dotenv

# load .env file
load_dotenv()

# read Twitch API credentials from .env file
twitch_client_id = os.getenv('TWITCH_CLIENT_ID')
twitch_client_secret = os.getenv('TWITCH_CLIENT_SECRET')

# prepare request
url = f'https://id.twitch.tv/oauth2/token?client_id={twitch_client_id}&client_secret={twitch_client_secret}&grant_type=client_credentials'

# send request for token
response = requests.post(url)

# extract token
access_token = response.json()['access_token']

In [7]:
twitch_api_url = 'https://api.igdb.com/v4'

headers = {
    'Client-ID': twitch_client_id,
    'Authorization': f'Bearer {access_token}'
}
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

In [8]:
def transform_platform_abbreviations(abbr):
    if abbr == '2600':
        return {'original': '2600', 'new': 'Atari2600'}
    if abbr == 'PS':
        return {'original': 'PS', 'new': 'PS1'}
    if abbr == 'XB':
        return {'original':'XB', 'new':'XBOX'}
    if abbr == 'GC':
        return {'original': 'GC', 'new': 'NGC'}
    if abbr == 'DS':
        return {'original': 'DS', 'new': 'NDS'}
    if abbr == 'GB':
        return {'original': 'GB', 'new':'Game Boy'}
    return {'original': abbr, 'new': abbr}

platform_abbreviations = df_games_with_missing_year['Platform'].unique()
platform_abbreviations_transform = np.vectorize(transform_platform_abbreviations)
platform_abbreviations_transormed = platform_abbreviations_transform(platform_abbreviations)
platform_abbreviations_transormed

array([{'original': 'PS2', 'new': 'PS2'},
       {'original': 'Wii', 'new': 'Wii'},
       {'original': '2600', 'new': 'Atari2600'},
       {'original': 'X360', 'new': 'X360'},
       {'original': 'GBA', 'new': 'GBA'}, {'original': 'PC', 'new': 'PC'},
       {'original': 'PS3', 'new': 'PS3'},
       {'original': 'PS', 'new': 'PS1'},
       {'original': 'PSP', 'new': 'PSP'},
       {'original': 'XB', 'new': 'XBOX'},
       {'original': 'GB', 'new': 'Game Boy'},
       {'original': 'DS', 'new': 'NDS'}, {'original': 'GC', 'new': 'NGC'}],
      dtype=object)

In [9]:
async def get_platform(abbreviation):
    url = f'{twitch_api_url}/platforms'
    async with aiohttp.ClientSession() as session:
        async with session.post(url, 
                                headers=headers, 
                                data=f'fields *; where abbreviation = "{abbreviation}";',
                                ssl=ssl_context) as response:
            return await response.json()

results = []

if os.path.exists('../files/platforms.pkl'):
    with open('../files/platforms.pkl', 'rb') as f:
        print('Loading from file')
        results = pickle.load(f)
else:
    print('fetching from Twitch API')
    tasks = [get_platform(abbr['new']) for abbr in platform_abbreviations_transormed]
    results = await asyncio.gather(*tasks)

results

Loading from file


[[{'id': 8,
   'abbreviation': 'PS2',
   'alternative_name': 'PS2',
   'category': 1,
   'created_at': 1297639288,
   'generation': 6,
   'name': 'PlayStation 2',
   'platform_logo': 254,
   'platform_family': 1,
   'slug': 'ps2',
   'updated_at': 1718027289,
   'url': 'https://www.igdb.com/platforms/ps2',
   'versions': [58, 114],
   'checksum': 'd0ab5fbe-eea9-026e-0c84-0d29fc53915e'}],
 [{'id': 5,
   'abbreviation': 'Wii',
   'alternative_name': 'Revolution',
   'category': 1,
   'created_at': 1297639288,
   'generation': 7,
   'name': 'Wii',
   'platform_logo': 326,
   'platform_family': 5,
   'slug': 'wii',
   'updated_at': 1718027289,
   'url': 'https://www.igdb.com/platforms/wii',
   'versions': [72, 283],
   'websites': [275],
   'checksum': 'aa433d9d-5fe2-ecb3-7822-536673adb8ee'}],
 [{'id': 59,
   'abbreviation': 'Atari2600',
   'alternative_name': 'Atari VCS',
   'category': 1,
   'created_at': 1372270420,
   'generation': 2,
   'name': 'Atari 2600',
   'platform_logo': 832,
 

In [10]:
# Persist results to a file to prevent re-running the code
with open ('../files/platforms.pkl', 'wb') as f:
    pickle.dump(results, f)

In [11]:
platform_id = pd.DataFrame([{'id': r[0]['id'], 'abbreviation': r[0]['abbreviation']} for r in results])

original = [item['original'] for item in platform_abbreviations_transormed]
new = [item['new'] for item in platform_abbreviations_transormed]

df_platform_abbreviations_transormed = pd.DataFrame({'original': original, 'new': new})
df_platform_abbreviations_transormed

merged_platforms_id = pd.merge(platform_id, df_platform_abbreviations_transormed, left_on='abbreviation', right_on='new')
merged_platforms_id.drop(columns=['new'], inplace=True)
merged_platforms_id

,id,abbreviation,original
0,8,PS2,PS2
1,5,Wii,Wii
2,59,Atari2600,2600
3,12,X360,X360
4,24,GBA,GBA
5,6,PC,PC
6,9,PS3,PS3
7,7,PS1,PS
8,38,PSP,PSP
9,11,XBOX,XB


In [12]:
def get_release_date(game_record):
    platform_id = merged_platforms_id[merged_platforms_id['original'] == game_record['Platform']]['id'].values[0]
    query = f'search "{game_record['Name']}"; fields first_release_date; where platforms = ({platform_id});'
    response = requests.post(f'{twitch_api_url}/games', headers=headers, data=query)
    response = response.json()
    if len(response) == 0:
        return None
    if 'first_release_date' not in response[0]:
        return None
    return datetime.datetime.fromtimestamp(response[0]['first_release_date']).year

if os.path.exists('../files/df_games_with_missing_year.pkl'):
    with open ('../files/df_games_with_missing_year.pkl', 'rb') as f:
        print('Reading df_games_with_missing_year from file')
        df_games_with_missing_year = pickle.load(f)
else:
    print('Fetching missing years for df_games_with_missing_year')
    for index, game_record in df_games_with_missing_year.iterrows():
        df_games_with_missing_year.at[index, 'Year'] = get_release_date(game_record)

df_games_with_missing_year
    

Reading df_games_with_missing_year from file


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
179,180,Madden NFL 2004,PS2,2003,Sports,Electronic Arts,4.26,0.26,0.01,0.71,5.23
377,378,FIFA Soccer 2004,PS2,2003,Sports,Electronic Arts,0.59,2.36,0.04,0.51,3.49
431,432,LEGO Batman: The Videogame,Wii,2008,Action,Warner Bros. Interactive Entertainment,1.86,1.02,0.00,0.29,3.17
470,471,wwe Smackdown vs. Raw 2006,PS2,2005,Fighting,NaN,1.57,1.02,0.00,0.41,3.00
607,608,Space Invaders,2600,2012,Shooter,Atari,2.36,0.14,0.00,0.03,2.53
624,625,Rock Band,X360,2007,Misc,Electronic Arts,1.93,0.34,0.00,0.21,2.48
649,650,Frogger's Adventures: Temple of the Frog,GBA,2001,Adventure,Konami Digital Entertainment,2.15,0.18,0.00,0.07,2.39
652,653,LEGO Indiana Jones: The Original Adventures,Wii,2008,Action,LucasArts,1.54,0.63,0.00,0.22,2.39
711,713,Call of Duty 3,Wii,2006,Shooter,Activision,1.19,0.84,0.00,0.23,2.26
782,784,Rock Band,Wii,2007,Misc,MTV Games,1.35,0.56,0.00,0.20,2.11


In [13]:
# transform year to int
df_games_with_missing_year['Year'] = df_games_with_missing_year['Year'].astype('Int64')

# save df with missing years to a file
with open('../files/df_games_with_missing_year.pkl', 'wb') as f:
    pickle.dump(df_games_with_missing_year, f)

In [14]:
df_games_with_missing_year.dropna(subset=['Year'], inplace=True)
df_games_with_missing_year

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
179,180,Madden NFL 2004,PS2,2003,Sports,Electronic Arts,4.26,0.26,0.01,0.71,5.23
377,378,FIFA Soccer 2004,PS2,2003,Sports,Electronic Arts,0.59,2.36,0.04,0.51,3.49
431,432,LEGO Batman: The Videogame,Wii,2008,Action,Warner Bros. Interactive Entertainment,1.86,1.02,0.00,0.29,3.17
470,471,wwe Smackdown vs. Raw 2006,PS2,2005,Fighting,NaN,1.57,1.02,0.00,0.41,3.00
607,608,Space Invaders,2600,2012,Shooter,Atari,2.36,0.14,0.00,0.03,2.53
624,625,Rock Band,X360,2007,Misc,Electronic Arts,1.93,0.34,0.00,0.21,2.48
649,650,Frogger's Adventures: Temple of the Frog,GBA,2001,Adventure,Konami Digital Entertainment,2.15,0.18,0.00,0.07,2.39
652,653,LEGO Indiana Jones: The Original Adventures,Wii,2008,Action,LucasArts,1.54,0.63,0.00,0.22,2.39
711,713,Call of Duty 3,Wii,2006,Shooter,Activision,1.19,0.84,0.00,0.23,2.26
782,784,Rock Band,Wii,2007,Misc,MTV Games,1.35,0.56,0.00,0.20,2.11


In [15]:
df_main.update(df_games_with_missing_year)

/var/folders/jk/596mp0_15870xtpjs5dl6qy00000gn/T/ipykernel_10276/4198323051.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<FloatingArray>
[2006.0, 1985.0, 2008.0, 2009.0, 1996.0, 1989.0, 2006.0, 2006.0, 2009.0,
 1984.0,
 ...
 2008.0, 2016.0, 2007.0, 2004.0, 2008.0, 2002.0, 2003.0, 2008.0, 2010.0,
 2003.0]
Length: 16598, dtype: Float64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_main.update(df_games_with_missing_year)


In [16]:
# as we can see, we have updated some Year columns
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16377 non-null  Float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: Float64(1), float64(5), int64(1), object(4)
memory usage: 1.4+ MB


In [17]:
# drop rows with na Year
df_main.dropna(subset=['Year'], inplace=True)

In [18]:
# drop rows with na Publisher
df_main.dropna(subset=['Publisher'], inplace=True)

In [19]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16339 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16339 non-null  int64  
 1   Name          16339 non-null  object 
 2   Platform      16339 non-null  object 
 3   Year          16339 non-null  Float64
 4   Genre         16339 non-null  object 
 5   Publisher     16339 non-null  object 
 6   NA_Sales      16339 non-null  float64
 7   EU_Sales      16339 non-null  float64
 8   JP_Sales      16339 non-null  float64
 9   Other_Sales   16339 non-null  float64
 10  Global_Sales  16339 non-null  float64
dtypes: Float64(1), float64(5), int64(1), object(4)
memory usage: 1.5+ MB


In [21]:
df_main['Year'] = df_main['Year'].astype('int64')

# Analysis

Now let's go to the analysis part of the project. In the following sections we will analyze
multiple sales of genres, regions and platforms across years.

We will also analyze what kind of games producers produce

## Genre sales analysis

First, let's do a simple analysis of genre sales. We can clearly see that the best selling genre is Action. It is\
followed by Sports and Shooter genres. TOP 3 genres sold 4097.99mln games in total, which accounts for 46% of overall game sales. If\
the publisher aims for high sales, they should definitely focus on these genres, but they also need to keep in mind, that these\
are the most competitive ones which means that to ahieve a success, one need to invest huge amount of money.

Next four best selling genres are rather evenly matched and maybe some closer more thourough investigation is needed\
to find what makes the Role Playing genre more appealing than Platform games.

In [39]:
genre_sales = df_main.groupby(['Genre'])['Global_Sales'].sum().sort_values(ascending=False)
print(genre_sales)
fig = px.bar(genre_sales, x=genre_sales.index, y=genre_sales.values, title='Global sales by genre', width=800, height=800)
fig.show()

Genre
Action          1741.82
Sports          1322.57
Shooter         1033.60
Role-Playing     925.44
Platform         830.16
Misc             799.95
Racing           729.89
Fighting         444.59
Simulation       389.98
Puzzle           242.95
Adventure        238.28
Strategy         173.90
Name: Global_Sales, dtype: float64


## Publisher sales

As we can see below, publisher which sells the most games is Nintendo, followed by Electronic Arts and Activision.

These three publishers alone account for 49.8% sales in total.

Sony Computer Entertainment and Ubisoft are strong competitors especially for Activision.

In [65]:
# first, let's find out which publishers sold over 100 mln copies of their games
publisher_sales = df_main.groupby(['Publisher'])['Global_Sales'].sum().sort_values(ascending=False)
publisher_sales = publisher_sales[publisher_sales > 100]
publisher_sales.index

Index(['Nintendo', 'Electronic Arts', 'Activision',
       'Sony Computer Entertainment', 'Ubisoft', 'Take-Two Interactive', 'THQ',
       'Konami Digital Entertainment', 'Sega', 'Namco Bandai Games',
       'Microsoft Game Studios', 'Capcom', 'Atari',
       'Warner Bros. Interactive Entertainment', 'Square Enix',
       'Disney Interactive Studios'],
      dtype='object', name='Publisher')

In [70]:
fig = px.bar(publisher_sales, x=publisher_sales.index, y=publisher_sales.values, title='Global sales by publisher', width=1200, height=800)
fig.show()

## Genre sales by producer

Now, let's analyse genre sales by publishers

In [91]:
genre_sales_by_producer = df_main[df_main['Publisher'].isin(publisher_sales.index)].groupby(['Publisher', 'Genre'])['Global_Sales'].sum().sort_values(ascending=False)
genre_sales_by_producer = genre_sales_by_producer.unstack().reset_index().melt(id_vars='Publisher', value_name='Global_Sales')
genre_sales_by_producer


,Publisher,Genre,Global_Sales
0,Activision,Action,141.82
1,Atari,Action,25.16
2,Capcom,Action,85.16
3,Disney Interactive Studios,Action,51.92
4,Electronic Arts,Action,115.34
...,...,...,...
187,Square Enix,Strategy,1.87
188,THQ,Strategy,7.49
189,Take-Two Interactive,Strategy,7.40
190,Ubisoft,Strategy,8.45


In [122]:
# count how many rows and cols we need for subplots
rows = len(genre_sales_by_producer['Genre'].unique())//3
cols = 4

fig = make_subplots(rows=rows, cols=cols, subplot_titles=genre_sales_by_producer['Genre'].unique())

for i, genre in enumerate(genre_sales_by_producer['Genre'].unique()):
    data = genre_sales_by_producer[genre_sales_by_producer['Genre'] == genre]
    fig.add_trace(go.Bar(x=data['Publisher'], y=data['Global_Sales'], name=genre), row=i//cols+1, col=i%cols+1)

fig.update_layout(height=2500, width=2200, title='Global genre sales by publisher')
fig.show()


1 1
1 2
1 3
1 4
2 1
2 2
2 3
2 4
3 1
3 2
3 3
3 4
